In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np

In [2]:
# Set a random seed for reproducibility
np.random.seed(42)

In [3]:
# Directory for Tiny ImageNet dataset
data_dir = "tiny-imagenet-200"

In [4]:
# Data transformations with augmentation for training and resizing for validation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(299),  # Resizing to 299 for Inception compatibility
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

val_transforms = transforms.Compose([
    transforms.Resize((299, 299)),  # Resizing to 299 for Inception compatibility
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


In [5]:
# Load the datasets
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

In [6]:
# Subset the training and validation datasets
train_subset_indices = np.random.choice(len(train_dataset), 20000, replace=False)
val_subset_indices = np.random.choice(len(val_dataset), 500, replace=False)

train_subset = Subset(train_dataset, train_subset_indices)
val_subset = Subset(val_dataset, val_subset_indices)

In [7]:
# Create DataLoaders
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False, num_workers=4)

In [8]:
print(f"Training subset data: {len(train_subset)} images")
print(f"Validation data: {len(val_subset)} images")

Training subset data: 20000 images
Validation data: 500 images


In [9]:
# EarlyStopping class
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [10]:
# Function to initialize pre-trained models

In [11]:
def initialize_model(model_name):
    if model_name == "VGG-19":
        model = models.vgg19(weights="IMAGENET1K_V1")
        model.classifier[6] = nn.Linear(4096, 200)
    elif model_name == "ResNet50V2":
        model = models.resnet50(weights="IMAGENET1K_V1")
        model.fc = nn.Linear(model.fc.in_features, 200)
    elif model_name == "InceptionV4":
        model = models.inception_v3(weights="IMAGENET1K_V1", aux_logits=True)  # Use InceptionV3 with aux_logits
        model.fc = nn.Linear(model.fc.in_features, 200)
    return model

In [12]:
# Training and validation functions

In [13]:
def train_one_epoch(model, data_loader, optimizer, criterion, device, is_inception=False):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in tqdm(data_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        # Handle auxiliary outputs if using Inception
        if is_inception:
            outputs, aux_outputs = model(images)
            loss = criterion(outputs, labels) + 0.4 * criterion(aux_outputs, labels)
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item()
        
    return running_loss / len(data_loader), 100 * correct / total

In [14]:
def validate(model, data_loader, criterion, device, is_inception=False):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Handle auxiliary outputs if using Inception and the model is in training mode
            if is_inception and model.training:
                outputs, _ = model(images)  # Only use the primary output during validation
            else:
                outputs = model(images)
                
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()
            
    return running_loss / len(data_loader), 100 * correct / total

In [15]:
# Training function with early stopping
def train_model(model_name, num_epochs=5):
    print(f"\nTraining {model_name} Model")
    
    model = initialize_model(model_name).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
    early_stopping = EarlyStopping(patience=3, min_delta=0.001)
    criterion = nn.CrossEntropyLoss()

    # Check if Inception model
    is_inception = model_name == "InceptionV4"
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device, is_inception)
        val_loss, val_acc = validate(model, val_loader, criterion, device, is_inception)
        
        print(f"{model_name} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping triggered for {model_name}.")
            break
        
        scheduler.step()

In [16]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
# Train each model

In [18]:
train_model("VGG-19", num_epochs=5)


Training VGG-19 Model
Epoch 1/5


100%|██████████| 313/313 [05:17<00:00,  1.02s/it]


VGG-19 - Train Loss: 5.2668, Train Acc: 0.72%, Val Loss: 5.9807, Val Acc: 0.00%
Epoch 2/5


100%|██████████| 313/313 [05:20<00:00,  1.02s/it]


VGG-19 - Train Loss: 5.1684, Train Acc: 1.26%, Val Loss: 5.3421, Val Acc: 7.00%
Epoch 3/5


...

VGG-19 - Train Loss: 5.1042, Train Acc: 1.67%, Val Loss: 6.0315, Val Acc: 3.40%
Epoch 4/5


100%|██████████| 313/313 [05:19<00:00,  1.02s/it]


VGG-19 - Train Loss: 5.0254, Train Acc: 2.04%, Val Loss: 5.6076, Val Acc: 4.00%
Epoch 5/5


100%|██████████| 313/313 [05:20<00:00,  1.02s/it]


VGG-19 - Train Loss: 4.9434, Train Acc: 2.69%, Val Loss: 6.4141, Val Acc: 1.80%
Early stopping triggered for VGG-19.


In [19]:
train_model("ResNet50V2", num_epochs=5)


Training ResNet50V2 Model
Epoch 1/5


100%|██████████| 313/313 [03:19<00:00,  1.57it/s]


ResNet50V2 - Train Loss: 5.0408, Train Acc: 2.71%, Val Loss: 6.3206, Val Acc: 0.40%
Epoch 2/5


100%|██████████| 313/313 [03:20<00:00,  1.56it/s]


ResNet50V2 - Train Loss: 4.4077, Train Acc: 8.05%, Val Loss: 6.8622, Val Acc: 1.40%
Epoch 3/5


100%|██████████| 313/313 [03:20<00:00,  1.56it/s]


ResNet50V2 - Train Loss: 4.0439, Train Acc: 12.63%, Val Loss: 8.0913, Val Acc: 0.40%
Epoch 4/5


100%|██████████| 313/313 [03:21<00:00,  1.56it/s]


ResNet50V2 - Train Loss: 3.7441, Train Acc: 17.21%, Val Loss: 9.2060, Val Acc: 1.00%
Early stopping triggered for ResNet50V2.


In [20]:
train_model("InceptionV4", num_epochs=5)


Training InceptionV4 Model
Epoch 1/5


100%|██████████| 313/313 [02:07<00:00,  2.46it/s]


InceptionV4 - Train Loss: 6.8252, Train Acc: 4.00%, Val Loss: 7.2661, Val Acc: 0.20%
Epoch 2/5


100%|██████████| 313/313 [02:06<00:00,  2.48it/s]


InceptionV4 - Train Loss: 5.5884, Train Acc: 12.71%, Val Loss: 8.1489, Val Acc: 0.60%
Epoch 3/5


100%|██████████| 313/313 [02:06<00:00,  2.47it/s]


InceptionV4 - Train Loss: 4.8353, Train Acc: 21.11%, Val Loss: 9.2087, Val Acc: 0.40%
Epoch 4/5


100%|██████████| 313/313 [02:06<00:00,  2.48it/s]


InceptionV4 - Train Loss: 4.1623, Train Acc: 29.81%, Val Loss: 9.8372, Val Acc: 1.20%
Early stopping triggered for InceptionV4.


In [21]:
# Assuming test data is stored in a directory named 'test' inside tiny-imagenet-200
test_dir = os.path.join(data_dir, 'test')

In [22]:
# Transformation for test data (same as validation transformations, no data augmentation)
test_transforms = transforms.Compose([
    transforms.Resize((299, 299)),  # Resizing to 299 for Inception compatibility
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [23]:
# Load the test dataset
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [24]:
print(f"Test data: {len(test_dataset)} images")

Test data: 10000 images


In [25]:
# Test function
def test(model, data_loader, criterion, device, is_inception=False):
    model.eval()  # Set model to evaluation mode
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():  # No gradients needed for testing
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Handle auxiliary outputs only if using Inception in training mode
            if is_inception and model.training:
                outputs, _ = model(images)  # Use both outputs only in training mode
            else:
                outputs = model(images)
                
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()
    
    avg_loss = running_loss / len(data_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [26]:
# Test each trained model
def evaluate_on_test_set(model_name):
    model = initialize_model(model_name).to(device)
    # If you saved model checkpoints, load them here:
    # model.load_state_dict(torch.load(f"{model_name}_best_model.pth"))

    criterion = nn.CrossEntropyLoss()
    is_inception = model_name == "InceptionV4"
    test_loss, test_acc = test(model, test_loader, criterion, device, is_inception)
    print(f"{model_name} - Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

In [27]:
# Example of calling evaluate_on_test_set for each model after training
evaluate_on_test_set("VGG-19")
evaluate_on_test_set("ResNet50V2")
evaluate_on_test_set("InceptionV4")

VGG-19 - Test Loss: 5.2231, Test Acc: 0.23%
ResNet50V2 - Test Loss: 5.3685, Test Acc: 0.00%
InceptionV4 - Test Loss: 5.2444, Test Acc: 0.06%
